### OFS with sparse gradient
- as given by "An online approach for feature selection for classification in big data"
- no implementation found




In [24]:
#`get mean of dataset
import pandas as pd 
df = pd.read_csv('datasets/binary/shuffle_spambase_normalized.csv')
df = df.drop(labels='class', axis=1)
ftr_means = df.mean()
total_mean = ftr_means.mean()


In [25]:
ftr_means

word_freq_make                0.023029
word_freq_address             0.014917
word_freq_all                 0.055031
word_freq_3d                  0.001528
word_freq_our                 0.031222
word_freq_over                0.016310
word_freq_remove              0.015709
word_freq_internet            0.009477
word_freq_order               0.017123
word_freq_mail                0.013169
word_freq_receive             0.022921
word_freq_will                0.056019
word_freq_people              0.016924
word_freq_report              0.005863
word_freq_addresses           0.011157
word_freq_free                0.012442
word_freq_business            0.019970
word_freq_email               0.020324
word_freq_you                 0.088645
word_freq_credit              0.004707
word_freq_your                0.072886
word_freq_font                0.007088
word_freq_000                 0.018651
word_freq_money               0.007541
word_freq_hp                  0.026380
word_freq_hpl            

In [29]:
import numpy as np

class OFSSGD:
    def __init__(self, reduction_threshold, reduction_value, n_total_ftrs, regularization_param, step_size):
        try:
            if len(reduction_threshold) == n_total_ftrs:
                self.vartheta = reduction_threshold
            else:
                raise ValueError("threshold vector and amount of features is not matching")
        except TypeError as e:
            self.vartheta = np.ones(n_total_ftrs) * reduction_threshold
        self.sigma = reduction_value
        self.W = np.zeros(n_total_ftrs)
        self.regularization_param = regularization_param
        self.step_size = step_size

    def __ola(self, x, y):
        # copied from ofs 
        if np.dot(x, self.W) * y <= 1: # should be 0, shouldn't it
            w_tilde = (1-self.regularization_param * self.step_size)*self.W + self.step_size * y * x
            w_hat = min(1, (1/np.sqrt(self.regularization_param)) / np.linalg.norm(w_tilde) )*w_tilde
            self.W = w_hat
        else:
            self.W *= (1-self.regularization_param*self.step_size)

    def __SGr(self):

        for i in range(len(self.W)):
            if self.W[i] > 0 and self.W[i] < self.vartheta[i]:
                self.W[i] = max(0, self.W[i] - self.sigma)
            elif self.W[i] < 0 and self.W[i] > -self.vartheta[i]:
                self.W[i] = min(0, self.W[i] + self.sigma)


    def train(self, x, y):
        self.__ola(x,y)
        self.__SGr()
            
    def get_weights(self):
        return np.where(self.W != 0)[0]    
        # i suppose that all featurs with weight unequal zero are kept

        

In [33]:
import numpy as np
from skmultiflow.data import FileStream
from skmultiflow.neural_networks import PerceptronMask
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

stream = FileStream('datasets/binary/shuffle_spambase_normalized.csv', target_idx=57)
stream.prepare_for_use()

x,y = stream.next_sample(batch_size=100)
predictor = PerceptronMask()
predictor.partial_fit(x,y, stream.target_values)

n_selected_ftr = 10

ofssgd = OFSSGD(reduction_threshold=ftr_means, reduction_value=0.02, regularization_param=0.01, step_size=0.2, n_total_ftrs=stream.n_num_features)

accuracy = []

for epoch in range(100):
    while stream.has_more_samples():
        # Load a new sample
        x, y = stream.next_sample(batch_size=10)

        # Select features
        for idx, label in enumerate(y):
            ofssgd.train(x[idx],label)

        selected_ftr = ofssgd.get_weights()
        # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
        x_reduced = np.zeros(x.shape)
        x_reduced[:, selected_ftr] = x[:, selected_ftr]

        # Test
        y_pred = predictor.predict(x)
        accuracy.append(accuracy_score(y, y_pred))

        # Train
        predictor.partial_fit(x, y)

    # Restart the FileStream
    stream.restart()

plt.plot(accuracy)
plt.show()
print("Amount of selected features: {}".format(len(selected_ftr)))

SyntaxError: invalid syntax (<ipython-input-33-e3b454a82be5>, line 20)

In [32]:
len

array([0.40117113, 0.1978384 , 1.0475191 , 0.14550818, 0.90270072,
       0.36858777, 0.74147135, 0.29915965, 0.36297699, 0.28299327,
       0.70040143, 0.70297065, 0.67321162, 0.16321719, 0.21210298,
       0.6261242 , 0.51100973, 0.56082907, 1.99902433, 0.19778532,
       1.73731596, 0.41654386, 0.42571194, 0.26678935, 0.        ,
       0.        , 0.        , 0.09270525, 0.        , 0.05329632,
       0.01392418, 0.        , 0.01598441, 0.        , 0.01281726,
       0.19160685, 0.2111128 , 0.01909121, 0.05385295, 0.16598072,
       0.        , 0.        , 0.0305919 , 0.01823236, 0.16360447,
       0.03414848, 0.00455268, 0.        , 0.09018661, 0.25676751,
       0.05301555, 0.32539079, 0.44422142, 0.12294531, 0.17915157,
       0.17304682, 0.44440257])